In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [33]:
data = pd.read_csv('GameLogs/game_logs.csv')
#处理130年大型棒球比赛数据

In [34]:
data.head()

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,...,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,na,1,FW1,na,1,0,...,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y
1,18710505,0,Fri,BS1,na,1,WS3,na,1,20,...,Asa Brainard,1.0,burrh101,Henry Burroughs,9.0,berth101,Henry Berthrong,8.0,HTBF,Y
2,18710506,0,Sat,CL1,na,2,RC1,na,1,12,...,Pony Sager,6.0,birdg101,George Bird,7.0,stirg101,Gat Stires,9.0,NaN,Y
3,18710508,0,Mon,CL1,na,3,CH1,na,1,12,...,Ed Duffy,6.0,pinke101,Ed Pinkham,5.0,zettg101,George Zettlein,1.0,NaN,Y
4,18710509,0,Tue,BS1,na,2,TRO,na,1,9,...,Steve Bellan,5.0,pikel101,Lip Pike,3.0,cravb101,Bill Craver,6.0,HTBF,Y


In [35]:
#date - Date of the game
#v_name - Visiting team game
#v_league - Visiting team league
#h_name - Home team name
#h_league - Visiting team league
#v_line_score - Visiting team line score, eg 010000(10)00
#park_id - ID of the park where the game was held
#attendance - Game atendance
#Pandas 会占用和数据框大小差不多的内存来节省时间。因为我们对准确度感兴趣，
#所以我们将 memory_usage 的参数设置为 ‘deep’，以此来获取更准确的数字。
data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171907 entries, 0 to 171906
Columns: 161 entries, date to acquisition_info
dtypes: float64(77), int64(6), object(78)
memory usage: 503.7 MB


In [44]:
for dtype in ['float','int64','object']:
    selected_dtype = data.select_dtypes(include=[dtype])
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
    mean_usage_mb = mean_usage_b/1024**2
    print 'Average memory usage for {} colums:{:03.5f}MB'.format(dtype,mean_usage_mb)

Average memory usage for float colums:1.29473MB
Average memory usage for int64 colums:1.12419MB
Average memory usage for object colums:4.99830MB


In [45]:
np.finfo('float64')

finfo(resolution=1e-15, min=-1.7976931348623157e+308, max=1.7976931348623157e+308, dtype=float64)

In [51]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else:
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b/1024**2
    return "{:03.4f}MB".format(usage_mb)
#计算深度数据内存

In [52]:
#整型数据处理
data_int = data.select_dtypes(include=['int64'])
converted_int = data_int.apply(pd.to_numeric, downcast='unsigned')

print mem_usage(data_int)
print mem_usage(converted_int)

compare_ints = pd.concat([data_int.dtypes,converted_int.dtypes],axis=1)
compare_ints.columns = ['before','after']
compare_ints.apply(pd.Series.value_counts)

7.0000MB
1.0000MB


,before,after
uint8,NaN,5.0
uint32,NaN,1.0
int64,6.0,NaN


In [53]:
#float数据处理
data_float = data.select_dtypes(include=['float'])
converted_float = data_float.apply(pd.to_numeric, downcast='float')

print mem_usage(data_float)
print mem_usage(converted_float)

converted_floats = pd.concat([data_float.dtypes,converted_float.dtypes],axis=1)
converted_floats.columns = ['before','after']
converted_floats.apply(pd.Series.value_counts)#这里计数的是 列-类型 的数目

100.0000MB
50.0000MB


,before,after
float32,NaN,77.0
float64,77.0,NaN


In [54]:
#查看现在数据内存情况
optimized_data = data.copy()
optimized_data[converted_int.columns] = converted_int
optimized_data[converted_float.columns] = converted_float
#object types
print mem_usage(data)
print mem_usage(optimized_data)

503.0000MB
446.0000MB


In [58]:
#object types优化
#NumPy 缺少对字符串值的支持。因为 Python 是一种高级的解释语言，它不能对数值的存储方式进行细粒度控制。
#这种限制使得字符串以分散的方式存储在内存里，不仅占用了更多的内存，而且访问速度较慢。
#对象列表中的每一个元素都是一个指针(pointer)，它包含了实际值在内存中位置的“地址”。

#存储在 Pandas 中的字符串的大小与作为 Python 中单独字符串的大小相同。
from sys import getsizeof
s1 = 'working out'
s2 = 'memory usage for'
s3 = 'strings in python is fun!'
s4 = 'strings in python is fun!'

for s in [s1,s2,s3,s4]:
    print getsizeof(s)
    
obj_series = pd.Series(['working out','memory usage for','strings in python is fun!','strings in python is fun!'])
obj_series.apply(getsizeof)

32
37
46
46


0    32
1    37
2    46
3    46
dtype: int64

In [60]:
#使用分类Categoricals,。category 类型在底层使用整数类型来表示该列的值，而不是原始值。Pandas 用一个单独的字典来映射整数值和相应的原始值之间的关系。当某一列包含的数值集有限时，这种设计是很有用的。
#当我们将列转换为 category dtype 时，Pandas 使用了最省空间的 int 子类型，来表示一列中所有的唯一值。
data_object = data.select_dtypes(include=['object']).copy()
data_object.describe()

,day_of_week,v_name,v_league,h_name,h_league,day_night,completion,forefeit,protest,park_id,...,h_player_6_id,h_player_6_name,h_player_7_id,h_player_7_name,h_player_8_id,h_player_8_name,h_player_9_id,h_player_9_name,additional_info,acquisition_info
count,171907,171907,171907,171907,171907,140150,116,145,180,171907,...,140838,140838,140838,140838,140838,140838,140838,140838,1456,140841
unique,7,148,7,148,7,2,116,3,5,245,...,4774,4720,5253,5197,4760,4710,5193,5142,332,1
top,Sat,CHN,NL,CHN,NL,D,"19820711,CHI11,5,5,54",H,V,STL07,...,grimc101,Charlie Grimm,grimc101,Charlie Grimm,lopea102,Al Lopez,spahw101,Warren Spahn,HTBF,Y
freq,28891,8870,88866,9024,88867,82724,1,69,90,7022,...,427,427,491,491,676,676,339,339,1112,140841


In [62]:
#选择了数据集中的第二列 day_of_week 来进行试验。
#在上面的表格中，我们可以看到它只包含了七个唯一的值。我们将使用 .astype() 的方法将其转换为 categorical。
data_week = data_object.day_of_week.astype('category')
print data_week.head()

0    Thu
1    Fri
2    Sat
3    Mon
4    Tue
Name: day_of_week, dtype: category
Categories (7, object): [Fri, Mon, Sat, Sun, Thu, Tue, Wed]


In [63]:
#Series.cat.codes 属性来返回 category 类型用来表示每个值的整数值。
data_week.head().cat.codes
#当对象列中少于 50% 的值时唯一对象时，我们应该坚持使用 category 类型。
#但是如果这一列中所有的值都是唯一的，那么 category 类型最终将占用更多的内存。
#这是因为列不仅要存储整数 category 代码，还要存储所有的原始字符串的值。

0    4
1    0
2    2
3    1
4    5
dtype: int8

In [64]:
print mem_usage(data_object.day_of_week)
print mem_usage(data_week)

4.0000MB
0.0000MB


In [65]:
#我们将编写一个循环程序，遍历每个对象列，检查其唯一值的数量是否小于 50%。如果是，那么我们就将这一列转换为 category 类型。
converted_obj = pd.DataFrame()
for col in data_object.columns:
    num_unique_values = len(data_object[col].unique())
    num_total_values = len(data_object[col])
    if num_unique_values/num_total_values < 0.5:
        converted_obj.loc[:,col] = data_object[col].astype('category')
    else:
        converted_obj.loc[:,col] = data_object[col]

In [66]:
print mem_usage(data_object)
print mem_usage(converted_obj)

compare_obj = pd.concat([data_object.dtypes,converted_obj.dtypes],axis=1)
compare_obj.columns = ['before','after']
compare_obj.apply(pd.Series.value_counts)

394.0000MB
39.0000MB


Exception TypeError: 'data type not understood' in 'pandas._libs.lib.array_equivalent_object' ignored


,before,after
object,78.0,NaN
category,NaN,78.0


In [77]:
#我们将所有对象列都转换为 category 类型，但是这种情况并不符合所有的数据集，因此务必确保事先进行过检查。
#此外，对象列的内存使用量已经从 752MB 将至 52MB，减少了 93%。
#现在，我们将其与数据框的其余部分结合起来，再与我们最开始的 861MB 的内存使用量进行对比。
#对datatime这一列进行优化，本来18860731，int32型，但有必要转换为datatime类型，int64内存大小
optimized_data[converted_obj.columns] = converted_obj
date = optimized_data.date
optimized_data['date']=pd.to_datetime(date, format='%Y-%m-%d')

print mem_usage(optimized_data)
optimized_data.date.head()

91.0000MB


0   1871-05-04 00:00:00
1   1871-05-05 00:00:00
2   1871-05-06 00:00:00
3   1871-05-08 00:00:00
4   1871-05-09 00:00:00
Name: date, dtype: datetime64[ns]

In [75]:
#输出列名:类型 -> key:value
dtypes = optimized_data.drop('date',axis=1).dtypes

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

columns_types = dict(zip(dtypes_col,dtypes_type))
print columns_types

{'v_left_on_base': 'float32', 'v_manager_id': 'category', 'v_strikeouts': 'float32', 'h_rbi': 'float32', '3b_umpire_name': 'category', 'v_hits': 'float32', 'v_caught_stealing': 'float32', 'hp_umpire_name': 'category', 'protest': 'category', 'h_name': 'category', 'v_stolen_bases': 'float32', 'lf_umpire_name': 'category', 'v_player_6_def_pos': 'float32', 'additional_info': 'category', 'v_score': 'uint8', 'h_player_3_name': 'category', 'completion': 'category', 'v_hit_by_pitch': 'float32', 'v_player_4_def_pos': 'float32', 'h_assists': 'float32', 'length_minutes': 'float32', 'h_player_7_def_pos': 'float32', 'v_at_bats': 'float32', 'h_balks': 'float32', 'h_player_4_name': 'category', 'h_starting_pitcher_name': 'category', 'h_pitchers_used': 'float32', 'v_triples': 'float32', 'v_player_2_def_pos': 'float32', 'h_player_9_def_pos': 'float32', 'h_player_3_id': 'category', 'v_player_3_def_pos': 'float32', 'v_player_4_name': 'category', 'h_walks': 'float32', 'number_of_game': 'uint8', 'h_player_5

In [76]:
optimized_data.head()

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,...,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,1871-05-04 00:00:00,0,Thu,CL1,na,1,FW1,na,1,0,...,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y
1,1871-05-05 00:00:00,0,Fri,BS1,na,1,WS3,na,1,20,...,Asa Brainard,1.0,burrh101,Henry Burroughs,9.0,berth101,Henry Berthrong,8.0,HTBF,Y
2,1871-05-06 00:00:00,0,Sat,CL1,na,2,RC1,na,1,12,...,Pony Sager,6.0,birdg101,George Bird,7.0,stirg101,Gat Stires,9.0,NaN,Y
3,1871-05-08 00:00:00,0,Mon,CL1,na,3,CH1,na,1,12,...,Ed Duffy,6.0,pinke101,Ed Pinkham,5.0,zettg101,George Zettlein,1.0,NaN,Y
4,1871-05-09 00:00:00,0,Tue,BS1,na,2,TRO,na,1,9,...,Steve Bellan,5.0,pikel101,Lip Pike,3.0,cravb101,Bill Craver,6.0,HTBF,Y
